In [1]:
import time
time.strftime("%c")

'Tue Oct  4 12:46:47 2016'

In [2]:
import os

from collections import defaultdict
from itertools import product

import pandas as pd
import requests

In [3]:
base = os.environ['BUG_FREE_EUREKA_BASE']
roster_path = os.path.join(base, 'grading', 'roster.tsv')

In [4]:
assignment_1 = ['DNA', 'RNA', 'GC', 'HAMM', 'REVC']
assignment_2 = ['INI', 'DBPR', 'FIB', 'IPRB']
assignment_3 = ['FRMT', 'PROT', 'SUBS', 'CONS', 'FIBD', 'GRPH', 'IEV', 'MPRT', 'MRNA', 'PERM', 'PRTM']
assignment_all = assignment_1 + assignment_2 + assignment_3

In [5]:
roster = pd.read_csv(roster_path, index_col=0, sep='\t')

# Checking GitHub

In [6]:
gh_check = requests.get('https://api.github.com/rate_limit')
gh_check_json = gh_check.json()

if gh_check_json['resources']['core']['remaining'] < len(roster.index):
    raise Exception('Not enough GitHub API calls left. Wait {}'.format(gh_check_json['resources']['core']['reset']))

In [7]:
gh_fmt = 'https://api.github.com/repos/{}/BioDb2016/contents/'

gh_result_dict = defaultdict(list)

for name, github_user, rosalind_user in roster.itertuples():
    print('Checking GitHub: {:12} https://github.com/{}'.format(name, github_user))
    
    response = requests.get(gh_fmt.format(github_user))
    
    for d in response.json():
        if 'type' not in d:
            raise Exception('Missing data.')
        if 'file' == d['type']:
            gh_result_dict[name].append(d['name'])

Checking GitHub: Akrishta     https://github.com/akrishta305
Checking GitHub: Alex         https://github.com/alexmasny
Checking GitHub: Apurva       https://github.com/ApurvaG05
Checking GitHub: Arunabh      https://github.com/arunabh99
Checking GitHub: Asma         https://github.com/AsmaTahir
Checking GitHub: Xin          https://github.com/huxin0221
Checking GitHub: Ihsan        https://github.com/ihsanmuchsin92
Checking GitHub: Amir         https://github.com/kayvanjoo
Checking GitHub: Lilit        https://github.com/lilith3
Checking GitHub: Ming         https://github.com/MingBit
Checking GitHub: Monica       https://github.com/monicaSudarsanam
Checking GitHub: Oyelle       https://github.com/oyelle
Checking GitHub: Randa        https://github.com/randaelsh
Checking GitHub: Sabby        https://github.com/sabyasachi2k13
Checking GitHub: Sakthi       https://github.com/Sakthi23
Checking GitHub: Sepehr       https://github.com/sepehrgolriz
Checking GitHub: Tamal        https://gith

In [8]:
gh_results = defaultdict(dict)

for name, files in gh_result_dict.items():
    filenames = [file.split('.')[0].upper() for file in files]
    for assignment in assignment_all:
        gh_results[name][assignment] = 1 if assignment in filenames else 0
    
gh_result_df = pd.DataFrame.from_dict(gh_results, orient='index')

# Checking Rosalind

In [9]:
rosalind_result_dict = defaultdict(dict)

for name, assignment in product(roster.index, assignment_all):
    rosalind_result_dict[name][assignment] = 0

In [10]:
for name, github, rosalind in roster.itertuples():
    print('Checking Rosalind: {:12} {}'.format(name, rosalind))

    res = requests.get(rosalind)

    for line in res.iter_lines():
        line = line.decode('utf-8').strip()
        if 'badge-success' not in line:
            continue
        success = line[1 + line.find('>'): line.find('<', 2)]
        rosalind_result_dict[name][success] = 1

Checking Rosalind: Akrishta     http://rosalind.info/users/akrishta/
Checking Rosalind: Alex         http://rosalind.info/users/alexmasny/
Checking Rosalind: Apurva       http://rosalind.info/users/ApurvaG05/
Checking Rosalind: Arunabh      http://rosalind.info/users/arunabh99/
Checking Rosalind: Asma         http://rosalind.info/users/AsmaTahir/
Checking Rosalind: Xin          http://rosalind.info/users/huxin0221/
Checking Rosalind: Ihsan        http://rosalind.info/users/ihsanm/
Checking Rosalind: Amir         http://rosalind.info/users/kayvanjo/
Checking Rosalind: Lilit        http://rosalind.info/users/lilit.antonyan/
Checking Rosalind: Ming         http://rosalind.info/users/MingBit/
Checking Rosalind: Monica       http://rosalind.info/users/monicaSudarsanam/
Checking Rosalind: Oyelle       http://rosalind.info/users/nazneen/
Checking Rosalind: Randa        http://rosalind.info/users/randaelsh/
Checking Rosalind: Sabby        http://rosalind.info/users/sabyasachi2k13/
Checking Ros

In [11]:
rosalind_result_df = pd.DataFrame(rosalind_result_dict).T
rosalind_result_df[rosalind_result_df.isnull()] = 0 # sanitize from missing assignments
rosalind_result_df = rosalind_result_df.astype(int)

# Summary

In [12]:
print('Assignment 1 has {} tasks'.format(len(assignment_1)))

pd.DataFrame({
    'GitHub': gh_result_df[assignment_1].sum(axis=1),
    'Rosalind': rosalind_result_df[assignment_1].sum(axis=1)
})

Assignment 1 has 5 tasks


,GitHub,Rosalind
Akrishta,5,5
Alex,5,5
Amir,5,5
Apurva,5,5
Arunabh,5,5
Asma,5,5
Hanan,5,5
Haritha,5,5
Ihsan,5,5
Lilit,5,5


In [13]:
print('Assignment 2 has {} tasks'.format(len(assignment_2)))

pd.DataFrame({
    'GitHub': gh_result_df[assignment_2].sum(axis=1),
    'Rosalind': rosalind_result_df[assignment_2].sum(axis=1)
})

Assignment 2 has 4 tasks


,GitHub,Rosalind
Akrishta,4,4
Alex,4,4
Amir,4,4
Apurva,4,4
Arunabh,4,4
Asma,3,4
Hanan,2,3
Haritha,3,3
Ihsan,4,4
Lilit,4,4


In [14]:
print('Assignment 3 has {} tasks'.format(len(assignment_3)))

pd.DataFrame({
    'GitHub': gh_result_df[assignment_3].sum(axis=1),
    'Rosalind': rosalind_result_df[assignment_3].sum(axis=1)
})

Assignment 3 has 11 tasks


,GitHub,Rosalind
Akrishta,8,9
Alex,9,9
Amir,11,9
Apurva,4,5
Arunabh,8,9
Asma,0,7
Hanan,4,4
Haritha,0,0
Ihsan,11,11
Lilit,6,7
